In [1]:
from tensorflow.keras.models import load_model

import librosa
import numpy as np

model1 = load_model("model_with_CNN-kernel=(3,3).h5")


In [2]:
def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    y, _ = librosa.effects.trim(y)
    y = librosa.util.normalize(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    
    if mfcc.shape[1] >= 100:
        mfcc = mfcc[:, :100]
    else:
        pad_width = 100 - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')

    return mfcc


In [3]:
def extract_true_label(filename):
    if '_' in filename:
        emotion = filename.split('_')[2].split('.')[0].lower()
        if emotion =='ps':
             return 'surprised'
        if emotion =='fearful':
                return 'fear'
        else:
             return emotion
    elif '-' in filename:
        number = filename.split('-')[2]
        emotion_map = {
            '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
            '05': 'angry', '06': 'fear', '07': 'disgust', '08': 'surprised'
        }
        return emotion_map.get(number)

In [4]:
from sklearn.preprocessing import LabelEncoder


def get_label_encoder():
    duygular = [
    'angry',
    'calm',
    'disgust',
    'fear',
    'happy',
    'neutral',
    'sad',
    'surprised'
]
    encoder = LabelEncoder()
    encoder.fit(duygular)
    return encoder

In [9]:
import os
import numpy as np
from sklearn.metrics import classification_report

def predict_and_evaluate_cnnlstm(folder_path, model, label_encoder):
    files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
    X = []
    y_true = []
    for file in files:
        file_path = os.path.join(folder_path, file)
        label = extract_true_label(file)
        mfcc = extract_mfcc(file_path)  
        X.append(mfcc)
        y_true.append(label)
    X = np.array(X)
    X = X[...,np.newaxis]
    y_true_encoded = label_encoder.transform(y_true)
    
    print(X.shape)

    y_pred_probs = model.predict(X)
    
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true_encoded = np.array(y_true_encoded)  

    labels = np.unique(y_true_encoded)
    target_names = label_encoder.inverse_transform(labels)

    print("İlk 10 Dosya")
    print(np.unique(y_pred))
    print(label_encoder.classes_)
    print(classification_report(y_true_encoded, y_pred, target_names=target_names))



In [10]:
encoder = get_label_encoder()
predict_and_evaluate_cnnlstm("ses/test_20",model1,encoder)

(20, 20, 100, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
İlk 10 Dosya
[0 2 3 4 5 6 7]
['angry' 'calm' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprised']
              precision    recall  f1-score   support

       angry       1.00      1.00      1.00         6
     disgust       1.00      1.00      1.00         4
        fear       1.00      1.00      1.00         1
       happy       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         2
         sad       1.00      1.00      1.00         5
   surprised       1.00      1.00      1.00         1

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

